### Imports

In [ ]:
from kub.course.simlib.simulation import FMUSimulation
from kub.course.plotlib.simulationPlotFactory import SimulationPlotFactory
database = "../../database/day1/fmus/"

### **Conduction: Exercise 1**

<ins>Boundary conditions:<ins>
- Exterior temperature of : 5°C (278.15 K)
- Internal flow at wall_resistor surface : 5 W
- Internal temperature at wall_conductor surface : 25°C (298.15 K)

<ins>Unknowns:<ins>
- Wall_resistor : R value
- Wall_conductor : U/G value


<ins>Expected result:<ins>
- Wall_resistor : surface temperature of 20°C
- Wall_conductor : Heat flux of 15 W

In [ ]:
sim1 = FMUSimulation(database + "Exercices_Conduction_SteStaWalResCon_01.fmu")

sim1.initialize()
sim1.initParameters( { "Wall_Resistor.R": 1.5, "Wall_Conductor.G": 0.75 } )
sim1.exitInitialization()

data1 = sim1.run( [ "Wall_Resistor.port_b.T", "Wall_Conductor.port_b.Q_flow" ] )

sim1Factory = SimulationPlotFactory()

sim1Factory.plot_subplots(
    time=data1["time"],
    data_type="raw",
    data_dict={ "Wall Resistor Temperature (°C)": data1["Wall_Resistor.port_b.T"] - 273.15,
                "Wall Conductor Heat Flow (W)": data1["Wall_Conductor.port_b.Q_flow"] },
    title="Exercise 1 — Output variables"
)

### **Conduction: Exercise 2**

<ins>Boundary conditions:<ins>

- Exterior temperature of : 5°C (278.15 K)
- wall_resistor R value : 3 K/W
- wall_conductor U value : 0.2 W/K

<ins>Unknowns :<ins>
- Interior Heat flow : Q_flow

<ins>Expected result :<ins>
- Room air : temperature of 20°C


In [ ]:
sim2 = FMUSimulation(database + "Exercices_Conduction_SteStaWalResCon_02.fmu")

sim2.initialize()
sim2.initParameters( { "InteriorHeatFlow.Q_flow": 8 } )
sim2.exitInitialization()

data2 = sim2.run( ["RoomAir.port.T"] )

sim2Factory = SimulationPlotFactory()

sim2Factory.plot_subplots(
    time=data2["time"],
    data_type="temperature",
    data_dict={"Room Air Temperature (°C)": data2["RoomAir.port.T"]},
    title="Exercise 2 — Room Air Temperature"
)

### **Conduction: Exercise 3**

For a wall of 1 square meter with the following materials :
- Wood :
    - 𝞴 : 0,13 W/m/K
    - e : 20 cm
    - l : 0,3 m
- Rammed :
    - 𝞴 : 0,75 W/m/K
    - e : 5 cm - 5 cm
    - l : 0,7 m - 1 m
- Straw : 
    - 𝞴 : 0,07 W/m/K
    - e : 15 cm
    - l : 0,7 m

We want an interior temperature of :
	20°C

**What is the heating power necessary to obtain such temperature ?**


In [ ]:
sim3 = FMUSimulation(database + "Exercices_Conduction_ComWalSys_03.fmu")

sim3.initialize()

wallParameters = { "wall_1.lambda": 0.75, "wall_1.e": 0.05, "wall_1.A": 0.7,
                   "wall_2.lambda": 0.07, "wall_2.e": 0.15, "wall_2.A": 0.7,
                   "wall_3.lambda": 0.13, "wall_3.e": 0.20, "wall_3.A": 0.3,
                   "wall_4.lambda": 0.75, "wall_4.e": 0.05, "wall_4.A": 0.7 }
sim3.initParameters( wallParameters )
sim3.initParameters( { "InteriorHeatFlow.Q_flow": 7.35 } )

sim3.exitInitialization()

data3 = sim3.run( ["wall_4.port_b.T", "InteriorHeatFlow.port.T"] )

sim3Factory = SimulationPlotFactory()

sim3Factory.plot_subplots(
    time=data3["time"],
    data_type="temperature",
    data_dict={"Room Air Temperature (°C)": data3["wall_4.port_b.T"]},
    title="Exercise 3 — Room Air Temperature"
)


### **Conduction: Exercise 4**

For two walls of 1 square meter with the following materials :

- Concrete :
    - 𝞴 : 0,13 W/m/K
    - e : 10 cm
    - C : ρ * cp * V : 0,1 * 2400 * 880
- Glass wool :
    - 𝞴 : 0,034 W/m/K
    - e : 10 cm
    - C : ρ * cp * V : 0,1 * 2400 * 880 : 

**What can you observe ?**

In [ ]:
sim4 = FMUSimulation(database + "Exercices_Conduction_ComWalSys_04.fmu")

sim4.initialize(startTime=0.0, stopTime=86400*30, timeStep=3600.0)
sim4.exitInitialization()

data4 = sim4.run( ["ExteriorInsulationTemperatureSensor.T", "InteriorInsulationTemperatureSensor.T",
                   "ExteriorTemperature.T"] )

sim4Factory = SimulationPlotFactory()

sim4Factory.plot_multi_curves(
    time=data4["time"],
    data_type="temperature",
    data_dict={ "Interior Temperature with Exterior Insulation (°C)": data4["ExteriorInsulationTemperatureSensor.T"],
                "Interior Temperature with Interior Insulation (°C)": data4["InteriorInsulationTemperatureSensor.T"],
                "Exterior Temperature (°C)": data4["ExteriorTemperature.T"] },
    title="Exercise 4 — Type of Insulation (Exterior vs Interior)"
)